In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import tikzplotlib
import os

# SPECTROGRAM ANN TYPES

In [ ]:
SPECTROGRAMS = ['spectrograms', 'mel_spectrograms', 'mfcc_spectrograms', 'cqt_spectrograms']
VARIABLES = ['n_fft', 'hop_length', 'win_length', 'n_mels', 'n_mfcc']

In [2]:
FILEPATH = "../IST non-JNB results/spectrogram_tensors"

In [31]:
columns = ['type', 'n_fft', 'hop_len', 'win_len', 'n_mel', 'n_mfcc', 'Accuracy', 'Epoch', 'Test Loss', 'F1', 'Recall', 'Precision', 'AUC-ROC', 'Confusion Matrix', 'Train Accuracy', 'Train Loss', 'Train F1', 'Train Recall', 'Train Precision', 'Train AUC-ROC', 'Train Confusion Matrix']
spectrograms = pd.DataFrame(columns=columns)

for folder in os.listdir(f"{FILEPATH}/mel_spectrograms"):
    if folder != ".DS_Store":
        # Load data
        test = pd.read_csv(f"{FILEPATH}/mel_spectrograms/{folder}/test.csv", index_col=None)
        train = pd.read_csv(f"{FILEPATH}/mel_spectrograms/{folder}/test.csv", index_col=None)

        test.drop(test.columns[0], axis=1, inplace=True)
        train.drop(train.columns[0], axis=1, inplace=True)

        # Find index of highest accuracy in test set
        idx_test = test['Accuracy'].idxmax()

        # Get row with highest accuracy in test set
        highest_test_row = test.loc[idx_test].copy()

        # Rename and drop columns in the test row
        highest_train_row = test.loc[idx_test].drop(columns=["Epoch"]).rename({"Accuracy": "Train Accuracy", "Test Loss": "Train Loss", "F1": "Train F1", "Recall": "Train Recall", "Precision": "Train Precision", "AUC-ROC": "Train AUC-ROC", "Confusion Matrix": "Train Confusion Matrix"}).copy()

        # Split folder name to extract information
        info = folder.split('-')
        data_info = {'type': 'mel', 'n_fft': info[0], 'hop_len': info[1], 'win_len': info[2], 'n_mel': info[3], 'n_mfcc': 0}

        # Assign values to additional columns
        for col, value in data_info.items():
            highest_test_row[col] = value
            highest_train_row[col] = value

        # Concatenate highest_test_row and highest_train_row along the column axis
        full_row = pd.concat([highest_test_row, highest_train_row], axis=1)


        # Append full_row to spectrograms DataFrame
        spectrograms = pd.concat([spectrograms, full_row])


for folder in os.listdir(f"{FILEPATH}/mfcc_spectrograms"):
    if folder != ".DS_Store":
        # Load data
        test = pd.read_csv(f"{FILEPATH}/mfcc_spectrograms/{folder}/test.csv", index_col=None)
        train = pd.read_csv(f"{FILEPATH}/mfcc_spectrograms/{folder}/test.csv", index_col=None)

        # Find index of highest accuracy in test set
        idx_test = test['Accuracy'].idxmax()

        # Get row with highest accuracy in test set
        highest_test_row = test.loc[idx_test].copy()

        # Rename and drop columns in the test row
        highest_train_row = test.loc[idx_test].drop(columns=["Epoch"]).rename({"Accuracy": "Train Accuracy", "Test Loss": "Train Loss", "F1": "Train F1", "Recall": "Train Recall", "Precision": "Train Precision", "AUC-ROC": "Train AUC-ROC", "Confusion Matrix": "Train Confusion Matrix"}).copy()

        # Split folder name to extract information
        info = folder.split('-')
        data_info = {'type': 'mfcc', 'n_fft': info[0], 'hop_len': info[1], 'win_len': info[2], 'n_mel': info[3], 'n_mfcc': info[4]}

        # Assign values to additional columns
        for col, value in data_info.items():
            highest_test_row[col] = value
            highest_train_row[col] = value

        # Concatenate highest_test_row and highest_train_row along the column axis
        full_row = pd.concat([highest_test_row, highest_train_row], axis=1)

        # Append full_row to spectrograms DataFrame
        spectrograms = pd.concat([spectrograms, full_row])


for folder in os.listdir(f"{FILEPATH}/cqt_spectrograms"):
    if folder != ".DS_Store":
        # Load data
        test = pd.read_csv(f"{FILEPATH}/cqt_spectrograms/{folder}/test.csv", index_col=None)
        train = pd.read_csv(f"{FILEPATH}/cqt_spectrograms/{folder}/test.csv", index_col=None)

        # Find index of highest accuracy in test set
        idx_test = test['Accuracy'].idxmax()

        # Get row with highest accuracy in test set
        highest_test_row = test.loc[idx_test].copy()

        # Rename and drop columns in the test row
        highest_train_row = test.loc[idx_test].drop(columns=["Epoch"]).rename({"Accuracy": "Train Accuracy", "Test Loss": "Train Loss", "F1": "Train F1", "Recall": "Train Recall", "Precision": "Train Precision", "AUC-ROC": "Train AUC-ROC", "Confusion Matrix": "Train Confusion Matrix"}).copy()

        # Split folder name to extract information
        info = folder.split('-')
        data_info = {'type': 'cqt', 'n_fft': 0, 'hop_len': info[0], 'win_len': 0, 'n_mel': 0, 'n_mfcc': 0}

        # Assign values to additional columns
        for col, value in data_info.items():
            highest_test_row[col] = value
            highest_train_row[col] = value

        full_row = pd.concat([highest_test_row, highest_train_row], axis=1)

        # Append full_row to spectrograms DataFrame
        spectrograms = pd.concat([spectrograms, full_row])

print(spectrograms)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects